In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

In [0]:
data = pd.read_csv("/content/drive/My Drive/PIAIC /cnn_news/bbc-text.csv")

In [0]:
data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [0]:
data.shape

(2225, 2)

In [0]:
data.head(7)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...


In [0]:
data["category"].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [0]:
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 1780
Test size: 445


In [0]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [0]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [0]:
test_text

1780    hobbit picture  four years away  lord of the r...
1781    game firm holds  cast  auditions video game fi...
1782    clarke plans migrant point scheme anyone plann...
1783    radcliffe will compete in london paula radclif...
1784    serena becomes world number two serena william...
                              ...                        
2220    cars pull down us retail figures us retail sal...
2221    kilroy unveils immigration policy ex-chatshow ...
2222    rem announce new glasgow concert us band rem h...
2223    how political squabbles snowball it s become c...
2224    souness delight at euro progress boss graeme s...
Name: text, Length: 445, dtype: object

In [0]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, \
                                              char_level=False)

In [0]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [0]:
x_test

array([[0., 1., 1., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [0]:
encoder = LabelEncoder()

encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)


In [0]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1780, 1000)
x_test shape: (445, 1000)
y_train shape: (1780, 5)
y_test shape: (445, 5)


In [0]:
batch_size = 32
epochs = 4
drop_ratio = 0.5

In [0]:
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Dense(212, activation = "relu"))
model.add(layers.Activation("relu"))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)

Train on 1602 samples, validate on 178 samples
Epoch 1/4
1602/1602 [==============================] - 2s 1ms/sample - loss: 0.3592 - acc: 0.8814 - val_loss: 0.1381 - val_acc: 0.9382
Epoch 2/4
1602/1602 [==============================] - 0s 89us/sample - loss: 0.0257 - acc: 0.9925 - val_loss: 0.1319 - val_acc: 0.9551
Epoch 3/4
1602/1602 [==============================] - 0s 86us/sample - loss: 0.0036 - acc: 1.0000 - val_loss: 0.1372 - val_acc: 0.9551
Epoch 4/4
1602/1602 [==============================] - 0s 84us/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 0.1348 - val_acc: 0.9607


In [0]:
history.history

{'acc': [0.88139826, 0.99250937, 1.0, 1.0],
 'loss': [0.3592295942063003,
  0.025685610070439714,
  0.003562330288670669,
  0.0014957661238195522],
 'val_acc': [0.93820226, 0.9550562, 0.9550562, 0.96067417],
 'val_loss': [0.13808259903714898,
  0.1318888274973698,
  0.1371713253889191,
  0.13477855845448677]}

In [0]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

445/445 [==============================] - 0s 77us/sample - loss: 0.1342 - acc: 0.9551


In [0]:
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])


Test Loss:  0.13416408353976988
Test Accuracy:  0.9550562


In [0]:
x_train[0].shape

(1000,)

In [0]:
#by using hyperprameter tunning

def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [0]:
batch_size = 16
epochs = 4
drop_ratio = 0.4
run_experiment(batch_size, epochs, drop_ratio)

batch size: 16, epochs: 4, drop_ratio: 0.4
	Test loss: 0.12980267733670353
	Test accuracy: 0.95730335


In [0]:
text_labels = encoder.classes_

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")  

hobbit picture  four years away  lord of the rings ...
Actual label:entertainment
Predicted label: entertainment

game firm holds  cast  auditions video game firm b ...
Actual label:tech
Predicted label: tech

clarke plans migrant point scheme anyone planning  ...
Actual label:politics
Predicted label: politics

radcliffe will compete in london paula radcliffe w ...
Actual label:sport
Predicted label: sport

serena becomes world number two serena williams ha ...
Actual label:sport
Predicted label: sport

ultimate game  award for doom 3 sci-fi shooter doo ...
Actual label:tech
Predicted label: tech

algeria hit by further gas riots algeria suffered  ...
Actual label:business
Predicted label: business

fast lifts rise into record books two high-speed l ...
Actual label:tech
Predicted label: business

muslim group attacks tv drama 24 a british muslim  ...
Actual label:entertainment
Predicted label: entertainment

us tv special for tsunami relief a us television n ...
Actual label:entertai

In [0]:
test_text.iloc[3]

'radcliffe will compete in london paula radcliffe will compete in the flora london marathon this year after deciding her schedule for 2005.  the 31-year-old won the race in 2002 on her marathon debut  defended her title 12 months later and will now seek a third title in the 17 april race.  it doesn t get any better than this for the 25th anniversary   said race director david bedford.  after announcing the greatest men s field ever we now have the greatest women s distance runner ever.  three years ago radcliffe smashed the women s world record in two hours 18 minutes 15 seconds.  the bedford star returned to london 12 months later  lowering her mixed-race world record of 2:17:18  which she set in chicago in october 2003  by one minute 53 secs. radcliffe s career took a setback when she failed to complete the olympic marathon and later dropped out of the athens 10 000m last august. but the 31-year-old bounced back to win the new york marathon in november. radcliffe  however  passed up 

In [0]:
prediction = model.predict(np.array([x_test[3]]))

In [0]:
prediction

array([[8.8620864e-06, 9.4388258e-03, 3.4760740e-06, 9.9051964e-01,
        2.9242252e-05]], dtype=float32)

In [0]:
predicted_label = text_labels[np.argmax(prediction)]
predicted_label

'sport'

In [0]:
#print(test_text.iloc[:50], "...")
print('Actual label:' + test_cat.iloc[3])
print("Predicted label: " + predicted_label + "\n") 

Actual label:sport
Predicted label: sport



Cycgct Dataset Model Training

In [0]:
# Assignment 4 - Bagging and Hyperparameter tuning  

<img src="./images/bags.jpg" alt="The bagging operation" width="400" align='center'/>